In [2]:
import numpy as np
import pandas as pd

from typing import Optional
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast, GPT2LMHeadModel
from fastai.text.all import *
import fastai
import re

c:\Users\cleo7\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 노래 데이터

In [23]:
lyrics1 = pd.read_csv("label_result_song_short.csv")
lyrics1.head()

,index,id,title,singer,genres,lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,2,분노


In [24]:
lyrics2 = pd.read_csv("translated_lyrics.csv")
lyrics2.head()

,index,id,title,singer,genres,lyrics,translated_lyrics
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변


In [25]:
lyrics = lyrics2.merge(lyrics1, on=["index", "id", "title", "singer", "genres", "lyrics"])
lyrics.head()

,index,id,title,singer,genres,lyrics,translated_lyrics,sentiment,sentiment_label
0,1,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Ay ay ay ay ay,Ay ay ay ay ay,2,분노
1,2,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,거울 속에 내 표정 봐 봐,거울 속에 내 표정 봐 봐,5,당황
2,3,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,느낌 So good 기다려온 D-day,느낌 그래서 좋아요. 기다려온 D-하루,0,기쁨
3,4,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,연습했던 손든 인사도 그대로 하면 돼,연습했던 손든 인사도 그대로 하면 돼,0,기쁨
4,5,37140709,첫 만남은 계획대로 되지 않아,TWS (투어스),댄스,Hairstyle check하고 한 번 Turn around,머리스타일 확인하고 한 번 회전 주변,2,분노


### 노래 데이터 최종

In [26]:
# 각 id에 대해 sentiment_label의 빈도를 계산하여, 가장 많이 나온 2개의 감정을 새로운 칼럼으로 추가

def get_top_two_sentiments(sentiments):
    sentiment_counts = sentiments.value_counts()
    # 가장 많은 2개 감정을 반환
    top_two = sentiment_counts.head(2).index.tolist()
    # 2개 미만일 경우 빈 값 처리
    return top_two + [None] * (2 - len(top_two))

# groupby와 agg를 활용해 변환
lyrics_final = lyrics.groupby("id").agg({
    "title": "first",   # 첫 번째 값 유지
    "singer": "first",  # 첫 번째 값 유지
    "genres": "first",  # 첫 번째 값 유지
    "lyrics": "first",  # 첫 번째 값 유지
    "translated_lyrics": list,  # 리스트로 묶음
    "sentiment_label": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
lyrics_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(lyrics_final['sentiment_label'].tolist(), index=lyrics_final.index)

# 불필요한 sentiment_label 열 삭제
lyrics_final = lyrics_final.drop(columns=["sentiment_label"])

lyrics_final.head()


,id,title,singer,genres,lyrics,translated_lyrics,top_sentiment,second_sentiment
0,418168,희재,성시경,"발라드, 국내영화",햇살은 우릴 위해 내리고,"[햇살은 우릴 위해 내리고 , 바람도 서롤 감싸게 했죠 , 우리 웃음속에, 계절은 오고 또 갔죠, 바람에 흔들리는 머릿결 , 내게 불어오는 그대 향기 , 예쁜 두 눈도 웃음 소리도 , 모두가 내 것이었죠, 이런 사랑 이런 행복 쉽다 했었죠, 이런 웃음 이런 축복, 내게 쉽게 올리 없죠, 눈물조차 울음조차, 닦지 못한 나, 정말로 울면, 내가 그댈 보내준 것 같아서, 그대 떠나가는 그 순간도 , 나를 걱정했었나요 , 무엇도 해줄 수 없는, 내 맘 앞에서, 그댄 나를 떠나간다 해도, 난 그댈 보낸 적 없죠, 여전히 그댄 나를 살게 하는 , 이율테니 , 이런 사랑 이런 행복 쉽다 했었죠 , 이런 웃음 이런 축복, 내게 쉽게 올리 없죠, 눈물조차 울음조차 닦지 못한 나, 정말로 울면, 내가 그댈 보내준 것 같아서, 그대 떠나가는 그 순간도 , 나를 걱정했었나요 , 무엇도 해줄 수 없는 내 맘 앞에서 , 그댄 나를 떠나간다 해도 , 난 그댈 보낸 적 없죠, 기다림으로 다시 시작일테니 , 얼마나 사랑했는지, 얼마나 또 울었는지, 그대여 한순간 조차 잊지 말아요 , 거기 떠나간 그 곳에서, 날 기억하며 기다려요, 하루씩 그대에게, 다가가는 나 일테니]",슬픔,기쁨
1,418598,친구라도 될 걸 그랬어,거미 (GUMMY),R&B/Soul,벌써 넌 내가 편하니,"[벌써 넌 내가 편하니, 웃으며 인사 할 만큼, 까맣게 나를 잊었니, 네 곁에 있는 사람 소개할만큼, 견디긴 너무 힘든데, 자꾸만 울고 싶은데, 내 옆이 아닌 자리에, 너를 보고 있는게, 왜 그게 행복한걸까, 친구라도 될 걸 그랬어, 모두 다 잊고서, 다른 사람 만나는, 널 보아도 슬프지 않게, 그저 바라보고 있었어, 한참동안 네 옆에 그 사람까지도, 잠시라도 더 보려고, 다시 혹시라도 널 보게되면, 그땐 모르는 척 해볼게, 웃어도 볼게 지금의 너처럼, 눈 감지 말고 보낼 걸, 가는 널 꼭 지켜볼 걸, 차가운 너의 걸음에, 마지막 내 눈물도, 묻혀서 보내버릴 걸, 너무 모진 너의 모습이, 미워져 버려서 다른 사람 만나는, 널 보아도 슬프지 않게, 그저 바라보고 있었어, 한참동안 네 옆에 그 사람까지도, 잠시라도 더 보려고, 다시 혹시라도 널 보게되면, 그땐 모르는 척 해볼게, 웃어도 볼게 지금의 너처럼, 차라리 잘된 것 같아, 다시 널 또 한번, 미워할 수 있을테니, 혹시 아직 너도 나처럼, 편해지지 못하고, 아파만 하는 거니, 애써 너도 참는거니, 혹시 네가 다시 돌아올까봐, 나의 곁은 아직 그대로, 비워져 있어 너의 자리라서]",불안,슬픔
2,711626,살다가,SG 워너비,발라드,살아도 사는 게 아니래,"[살아도 사는 게 아니래, 너 없는 하늘에, 창 없는 감옥 같아서, 웃어도 웃는 게 아니래, 초라해 보이고, 우는 것 같아 보인대, 사랑해도 말 못 했던 나, 내색조차 할 수 없던 나, 나 잠이 드는 순간조차, 그리웠었지, 살다가 살다가 살다가 너 힘들 때, 나로 인한 슬픔으로, 후련할 때까지, 울다가 울다가 울다가 너 지칠 때, 정 힘들면 단 한 번만 기억하겠니, 살다가, 웃어도 웃는 게 아니래, 초라해 보이고, 우는 것 같아 보인대, 사랑해도 말 못 했던 나, 내색조차 할 수 없던 나, 나 잠이 드는 순간조차, 그리웠었지, 살다가 살다가 살다가 너 힘들 때, 나로 인한 슬픔으로, 후련할 때까지, 울다가 울다가 울다가 너 지칠 때, 정 힘들면 단 한 번만 기억하겠니, 우린 마지못해 웃는 거겠지, 우린 마지못해 살아가겠지, 내 곁에 있어도 나의 곁에 있어도, 눈물나니까, 살다가 살다가 살다가 너 힘들 때, 나로 인한 슬픔으로, 후련할 때까지, 태워도 태워도 태워도 남았다면, 남김없이 태워도 돼, 후련할 때까지, 나 살다가, 나 살다가]",슬픔,불안
3,1500196,내사람,SG 워너비,R&B/Soul,내 가슴속에 사는 사람 내가 그토록 아끼는 사람,"[내 가슴속에 사는 사람 내가 그토록 아끼는 사람 , 너무 소중해 마음껏 안아보지도 못했던 , 누구에게나 흔한 행복 한 번도 준적이 없어서 , 맘 놓고 웃어본 적도 없는 그댈 사랑합니다 , 내가 기쁠 때나 슬플 때나 함께 울고 웃어주던 , 그댈 위해 내가 할 수 있는건 뭐든 해주고 싶어 , 안녕 내 사랑 그대여 이젠 내가 지켜줄게요 , 못난 날 믿고 참고 기다려줘서 고마워요 , 안녕 내 사랑 그대여 영원토록 사랑할게요 , 다시 태어나서 사랑한대도 그대이고 싶어요 , 어두운 밤길을 잃어도 서로 등불이 되어주고 , 비바람 몰아쳐도 지금 잡은 두 손 놓지 말아요 , 내가 힘들 때나 아플 때나 내 곁에 있어준 그대 , 미안하단 말로 고맙단 말을 대신 하던 나였죠 , 안녕 내 사랑 그대여 이젠 내가 지켜줄게요 , 못난 날 믿고 참고 기다려줘서 고마워요 , 안녕 내 사랑 그대여 영원토록 사랑할게요 , 다시 태어나서 사랑한대도 그대이고 싶어요 , 넘어지고 몇 번을 다시 넘어진다 해도 , 그대만 있다면 다시 일어날 수 있는데 , < 간주 , 안녕 내 사랑 그대여 이젠 내가 지켜줄게요 , 못난 날 믿고 참고 기다려줘서 고마워요 , 안녕 내 사랑 그대여 이렇게 그댈...",기쁨,불안
4,1854856,라라라,SG 워너비,발라드,그대는 참 아름다워요,"[그대는 참 아름다워요, 밤 하늘의 별빛보다 빛나요, 지친 나의 마음을 따뜻하게 감싸줄, 그대 품이 나의 집이죠, 세찬 바람 앞에서, 꺼질듯한 내 사랑도, 잘 참고서 이겨내줬어요, 정말 눈물나도록 고마운 맘 아나요, 그대 내곁에 살아줘서, 사랑해요 사랑해요, 내가 그대에게 부족한걸 알지만, 세월을 걷다보면 지칠때도 있지만 , 그대의 쉴곳이 되리라, 사랑해요 고마운 내 사랑, 평생 그대만을 위해 부를 이노래, 사랑 노래 함께 불러요, 둘이서 라라라, 그대 쳐진 어깨가 내맘을 아프게해요, 잘 해준것도 없는 나라서, 그대의 고운 손이 세월에 변했어요, 못지켜줘서 미안해요, 사랑해요 사랑해요, 내가 그대에게 부족한걸 알지만, 세월을 걷다보면 지칠때도 있지만, 그대의 쉴곳이 되리라, 사랑해요 고마운 내 사랑, 평생 그대만을 위해 부를 이노래, 사랑노래 함께 불러요, 둘이서 라라라, 고마워요 고마워요, 그대 자신보다 나를 아껴준 사랑, 세상이 등 돌려도 누가 뭐라고 해도, 내가 그대 지켜줄게요, 사랑해요 소중한 내사랑, 평생 그대만을 위해 부를 이노래, 사랑노래 함께 불러요, 둘이서 라라라, 그대 품이 나의 집이죠 영원히 라라라]",기쁨,슬픔


In [27]:
lyrics_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755 entries, 0 to 754
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 755 non-null    int64 
 1   title              755 non-null    object
 2   singer             755 non-null    object
 3   genres             755 non-null    object
 4   lyrics             755 non-null    object
 5   translated_lyrics  755 non-null    object
 6   top_sentiment      755 non-null    object
 7   second_sentiment   755 non-null    object
dtypes: int64(1), object(7)
memory usage: 47.3+ KB


## 소설 데이터

In [12]:
novel = pd.read_csv("classified_novel1.csv")
novel.head()

,Column1,file_id,title,paragraph_id,text,labels,sentiment
0,0,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.1,01범보다 무서운 곶감,4.0,불안
1,1,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.2,화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다.,4.0,불안
2,2,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.3,"그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어, 사람이.",2.0,분노
3,3,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.4,"그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여.",4.0,불안
4,4,WARW1800000007,이야기꾼 구연설화,WARW1800000007.1.5,"애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우, ‘괭이 왔다’두 울구, 왼갖 소리를 다 해두 울어.",1.0,슬픔


In [20]:
novel_final = novel.groupby("file_id").agg({
    "title": "first",
    "text": list,  # 리스트로 묶음
    "sentiment": lambda x: get_top_two_sentiments(x)  # 빈도가 높은 2개의 감정 추출
}).reset_index()

# sentiment_label에서 두 개의 감정을 각각의 칼럼으로 분리
novel_final[['top_sentiment', 'second_sentiment']] = pd.DataFrame(novel_final['sentiment'].tolist(), index = novel_final.index)

# 불필요한 sentiment_label 열 삭제
novel_final = novel_final.drop(columns=["sentiment"])

novel_final.head()

,file_id,title,text,top_sentiment,second_sentiment
0,WARW1800000007,이야기꾼 구연설화,"[01범보다 무서운 곶감, 화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다., 그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어, 사람이., 그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여., 애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우, ‘괭이 왔다’두 울구, 왼갖 소리를 다 해두 울어., 그랑깨 꼭감을,, “아나, 꼭감.”, “아가, 꼭감.”, 그랑깨 꺄. 안 울어., 호랭이 생각이? ‘야아, 이거 내가 젤 무선 짐승인디 말여? 나보다 무서웅 게 익구나. 호랭이, 꼭감이, 꼭갬이 참 무석구나.’ 싶어서, 인제…, 근디 인제, 그 집이 말여....",상처,분노
1,WARW1800000008,편집국 쪽으로 : 소설로 읽는 편집기자 24시,"[술 먹다말고, 호외 찍다, “박 기자, 김 기자, 그만 마시고 회사로 돌아가자. 사건이 터졌대.”, 세 번째 오백 잔을 반쯤 비웠나 싶은데, 앞쪽에 앉은 이인철 차장이 가방을 주섬주섬 손에 들었다., “계산서 어디 있지?”, 편집기자가 급히 필요하다는 내용으로 봐서, 방금 전 전화는 회사에서 걸려온 것인 듯했다., “선배, 아직 안주가 많이 남았잖아요.”, 박수미 기자의 말엔 아쉽다는 느낌이 역력했다. 하지만 그녀도 황급한 술자리 종결이 싫지만은 않은 듯했다. 계산서 들고 호프집 입구로 향하는 이 차장을 총알같이 따라나서는 데선 약간의 흥분도 감지됐다., “박근혜가 테러를 당했대. 유세장에서 누가 갑자기 나타나서 얼굴을 그었다네. 박 기자 휴대폰 디엠비폰이지? 뉴스 좀 틀어봐.”, 이 차장은 택시에 자리 잡고 나서야 먹는 둥 마는 둥 술집을 나온 배경을 설명했다., “호외를 찍는다고 하네. 오전에 우리가 넘긴 본판은 이미 배달까지 끝난 상태라 개판이 힘들대. 박 기자, 휴대폰 좀 줘볼래?”, 그리곤 아무 말 없이 디엠비 방송에 얼굴을 파묻는 이 차장의 행동은 자못 진지했다., 하지만 예준은 사정이 달랐다. 편집기자 4년차에 처음 맞는 호외 발행. 하...",불안,기쁨
2,WARW1800000013,석각의 비밀 : 김제철 소년소설,"[석각에는 어떤 비밀이 숨어 있는걸까?, 잠시 창준은 저만치 수면 위로 드러나 있는 바위를 바라보았다., 그리고는, “자, 저기를 돌아오기다!”, 영규를 돌아보며 의미심장한 눈빛을 보냈다. 50미터는 족히 돼 보이는 거리였다., “좋아.”, 영규가 입가에 가벼운 웃음을 띤 채 대답했다. 여유 있는 표정이 었다., “그럼 셋에 출발한다. 하나, 둘, 셋!”, 힘차게 소리치며 창준이 몸을 날렸다. 동시에 영규도 물속으로 몸을 던졌다., 창준은 어깨를 바쁘게 움직여 양팔을 교대로 교차하며 물을 긁었다. 그리고 속도를 높이기 위해 두 발로는 계속해서 물장구를 쳤다. 몸과 물이 일체가 되었다고 느껴지자 창준은 더욱 부지런히 어깨를 움직였다. 생각 이상으로 빠르게 몸이 앞으로 나아갔다., 그동안 집중적으로 연습을 한 게 바로 이 크롤, 즉 자유형 영법이었다. 전엔 평형을 주로 했었지만 자유형을 처음 연습하면서부터 창준은 이 영법이 마음에 들었다. 그래서 그 후로 줄곧 크롤을 숙달하는 데 노력을 기울였다., 바위가 가까워졌을 때 창준은 몸을 뒤틀고 얼굴을 돌려 숨을 쉬며 옆을 보았다. 영규도 비슷한 지점까지 와 있었다. 그런데 영규는 여전히 평형을 고수하고 있었다...",불안,기쁨
3,WARW1800000016,곰팡이 빵,"[드르륵., 영우는 책상 서랍을 열다가 우뚝 멈추었다., 까맣게 곰팡이가 핀 앙고 빵 때문이었다., 영우는 곰팡이 빵을 뚫어지게 보았다., 눈이 시큰거렸다., 영우는 팔뚝으로 눈가를 쓱 문질렀다., 매미가 시끄럽게 울던 날, 삼촌은 할머니 손에 이끌려 영우 집에 왔다., 주춤주춤 삼촌이 할머니 뒤로 숨었다., 할머니가 가시자 삼촌은 아빠 뒤로 숨었다., “삼촌 잘 데리고 놀아라.”, 아빠가 말했다., “삼촌이면 어른이야. 나이 어리다고 함부로 하면 못 써. 너도 혼자라서 심심했을 텐데 잘 됐다.”, 엄마도 거들었다., 삼촌은 엄마 뒤로 다시 엉거주춤 몸을 뺐다., ‘치, 누가 잡아먹기라도 하나. 잘 데리고 놀라며 함부로 하지는 말라니. 그런 말이 어디 있어?’, 영우는 어린 삼촌이 못마땅했다., “엄마, 저 애한테 정말 삼촌이라고 해야 해?”, “옛날에는 아무리 어려도 촌수가 높으면 존댓말을 썼어. 지금이야 그럴 필요까진 없지만 그래도 삼촌은 삼촌이야. 어른들 앞에서는 절대 이름 부르지 말고 삼촌이라고 불러. 알았지?”, 엄마도 영우 편이 아니었다., 영우는 삼촌이 괜스레 얄미워졌다., “야! 물 좀 떠와.”, “야, 부채 갖고 와.”, 영우는 일부러...",불안,당황
4,WARW1800000017,담배 한 개비의 시간,"[담배 한 개비의 시간, 첫번째 기억, 나를 가졌을 때 우리 엄마는, 먹고 싶은 것이 하나도 없었다고 한다. 큰언니를 가졌을 땐 다량의 에이스크래커를, 둘째언니를 가졌을 땐 1.5톤 트럭 분량의 단팥 고로케를 필요로 했던 엄마의 몸은, 나를 위해서는 아무것도 필요로 하지 않았다., 아니, 어쩌면 그것은 엄마 몸의 필요보다는 단지 언니들 취향의 문제일 수도 있겠지만, 여하튼 그때 나는 아무것도 원하지 않았다., 왜냐하면 그 무렵 우리 엄마는 아주 깊은 슬픔에 빠져 있었기 때문이다. 내가 태어나기 얼마 전, 엄마의 엄마가 돌아가셨다. 엄마는 할머니를 마음속 깊이 사랑했다. 깊은 밤 엄마가 조용히 눈물을 흘릴 때면, 나도 빠짐없이 따라 울었다. 그래서 나를 감싸고 있던 양수의 염도는 조금씩 조금씩 높아져갔다., 어쩌면 나는 엄마의 슬픔을 양분으로 삼아 자라났는지도 모른다. 그런 의미에서 나를 구성하는 세포들은, 모두 슬픔이라는 핵을 그 안에 하나씩 지니고 있을 것이다., *, “양파를 썰 때 나오는 눈물을 막기 위해서는 양파조각 하나를 머리 위에 얹어놓는 것이 좋아요.”, 한 여자가 코를 훌쩍이며 양파 한 조각을 모자처럼 머리 위에 얹는다. 그리고 손등으로...",불안,기쁨


In [27]:
novel_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_id           375 non-null    object
 1   title             375 non-null    object
 2   text              375 non-null    object
 3   top_sentiment     375 non-null    object
 4   second_sentiment  375 non-null    object
dtypes: object(5)
memory usage: 14.8+ KB


In [21]:
novel_final.to_csv("novel_final.csv")

## KoGPT2

### 사전 테스트

In [ ]:
## 이제부턴 이걸로.
#lyrics_final = pd.read_csv("lyrics_final.csv")         이상하게 이렇게 하면 가사 부분 합치는 대서 결과가 달라짐,, 
novel_final = pd.read_csv("novel_final.csv")

In [22]:
# KoGPT2 기본 모델 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [28]:
novel_final["top_sentiment"].unique()

array(['상처', '불안', '기쁨', '슬픔', '분노', '당황'], dtype=object)

In [16]:
# 감정 및 노래 가사 설정
top_sentiment = "슬픔"
second_sentiment = "기쁨"
lyrics = " ".join(lyrics_final["translated_lyrics"][0])
lyrics

'햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [31]:
# 입력 텍스트 구성
input_text = f"{top_sentiment} | {second_sentiment} | {lyrics}"
input_text

'슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니'

In [32]:
# 토큰화
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# 문장 생성
gen_ids = model.generate(
    input_ids,
    max_length=500,  # 최대 생성 길이 설정
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True
)

# 결과 디코딩
generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
print(generated_text)

슬픔 | 기쁨 | 햇살은 우릴 위해 내리고  바람도 서롤 감싸게 했죠  우리 웃음속에 계절은 오고 또 갔죠 바람에 흔들리는 머릿결  내게 불어오는 그대 향기  예쁜 두 눈도 웃음 소리도  모두가 내 것이었죠 이런 사랑 이런 행복 쉽다 했었죠 이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서 그댄 나를 떠나간다 해도 난 그댈 보낸 적 없죠 여전히 그댄 나를 살게 하는  이율테니  이런 사랑 이런 행복 쉽다 했었죠  이런 웃음 이런 축복 내게 쉽게 올리 없죠 눈물조차 울음조차 닦지 못한 나 정말로 울면 내가 그댈 보내준 것 같아서 그대 떠나가는 그 순간도  나를 걱정했었나요  무엇도 해줄 수 없는 내 맘 앞에서  그댄 나를 떠나간다 해도  난 그댈 보낸 적 없죠 기다림으로 다시 시작일테니  얼마나 사랑했는지 얼마나 또 울었는지 그대여 한순간 조차 잊지 말아요  거기 떠나간 그 곳에서 날 기억하며 기다려요 하루씩 그대에게 다가가는 나 일테니 언제나 너를 그리워해요
그대 곁에 있어줘서 고마워요, 그리고 이제부터라도 널 만나고 싶어서 더 이상 미안한 마음 없어요.
사랑하는 나의 모든 것을 다 안고 가겠어요,
그리고 지금껏 당신을 기다리고 있는 당신의 모습을 영원히 잊을 거예요. 오늘 아침신문 보시죠.
먼저 조선일보입니다.
오늘 오전 10시부터 서울 광화문광장에서 '박근혜 퇴진 비상국민행동'이 주최로 열리는 집회가 열렸습니다.
집회 참가자들은 박 대통령의 즉각적인 하야와 국정 정상화를 촉구하고 있습니다.
이들은 "대통령의 사과와 책임자 처벌, 대통령 권한대행 체제 전환" 등을 요구하면서 청문회와 특검 도입 등 검찰 개혁과 국정조사를 요구하고 나섰는데 특히 새누리당 비주류 의원들이 탄핵에 동참할 움직임을 보이고 있다고 전했는데.
새정치민주연합 문재인 대표는 어제 최고위원회의에서 "이번 주말 촛불 민심은 새로운 대한민국을 향한 

- 사전학습된 자료가 뉴스라든가 그런거라서 이렇게 이상하게 나온다..

### Fine-Tuning

https://velog.io/@yeah7598/KoGPT2-%EB%8F%99%ED%99%94-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B0

In [ ]:
## 이제부턴 이걸로.
novel_final = pd.read_csv("novel_final.csv")

ParserError: Error tokenizing data. C error: Expected 50 fields in line 7, saw 100


- 여기 참고해서 이어서 ㄱㄱ

In [22]:
data = novel_final.copy()

In [23]:
data.columns

Index(['Unnamed: 0', 'file_id', 'title', 'text', 'top_sentiment',
       'second_sentiment'],
      dtype='object')

- 감정 : '상처', '불안', '기쁨', '슬픔', '분노', '당황'

In [24]:
from sklearn.preprocessing import LabelEncoder

# 감정 카테고리
emotions = ['상처', '불안', '기쁨', '슬픔', '분노', '당황']

# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

LabelEncoder()

In [25]:
# 감정1과 감정2를 숫자로 변환
data['감정1_encoded'] = le.transform(data['top_sentiment'])
data['감정2_encoded'] = le.transform(data['second_sentiment'])

# 수치형 감정1, 감정2와 텍스트 결합
data['input_text'] = data['감정1_encoded'].astype(str) + " | " + data['감정2_encoded'].astype(str) + " | " + data['text']

data['input_text'].head()

0    4 | 2 | ['01범보다 무서운 곶감', '화자를 처음 만나 이야기를 들으러 왔다고 하자 서슴없이 꺼낸 첫 이야기이다. 화자로서 가장 쉽게 기억해낸 이야기인 셈이다. 설화 앞뒤에 교훈적 해석을 덧붙이고 있음은 화자의 습관화된 태도의 한 모습이기도 하다. 어려서 조모로부터 들었다고 했다.', '그링깨. 사람이 어거지루는 못 살구. 응? 어거지루 안 되능 거여. 사람이 그링깨 뭐이냐 하먼 자연~간 제절루 되야지 어거지루는 못 살어, 사람이.', '그래 옛날, 그 꼭감이라능 게 말여. 사람이 먹잖야 이케? 먹지마는. 그게 참 무성(무서운) 거여.', '애기가 울어. 옛날에. 그래 할머니가 달갸(달래). 그때 호랭이가, 응? 그 집 문앜이 와 섰어 지금. 옛날이는 호랭이가 말두 하구 그랴. 그래 인제 그 할머니가 애기를 달램성 왼갖 소리를 다 햐. ‘호랭이 왔다’구 해두 울구우, ‘괭이 왔다’두 울구, 왼갖 소리를 다 해두 울어.', '그랑깨 꼭감을,', '“아나, 꼭감.”', '“아가, 꼭감.”', '그랑깨 꺄. 안 울어.', '호랭이 생각이? ‘야아, 이거 내가 젤 무선 짐승인디 말여? 나보다 무서웅 게 익구나. 호랭이, 꼭감이, 꼭갬이 참 무석구나....
1    3 | 0 | ['술 먹다말고', '호외 찍다', '“박 기자, 김 기자, 그만 마시고 회사로 돌아가자. 사건이 터졌대.”', '세 번째 오백 잔을 반쯤 비웠나 싶은데, 앞쪽에 앉은 이인철 차장이 가방을 주섬주섬 손에 들었다.', '“계산서 어디 있지?”', '편집기자가 급히 필요하다는 내용으로 봐서, 방금 전 전화는 회사에서 걸려온 것인 듯했다.', '“선배, 아직 안주가 많이 남았잖아요.”', '박수미 기자의 말엔 아쉽다는 느낌이 역력했다. 하지만 그녀도 황급한 술자리 종결이 싫지만은 않은 듯했다. 계산서 들고 호프집 입구로 향하는 이 차장을 총알같이 따라나서는 데선 약간의 흥분도 감지됐다.', '“박근혜가 테러를 당했대. 유세장에서 누가 갑자기 나타나서 얼굴을 그었다네. 박 기자 휴대

In [26]:
from transformers import PreTrainedTokenizerFast, AutoModelWithLMHead
import torch

# KOGPT2 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
c:\Users\cleo7\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\auto\modeling_auto.py:1881: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [27]:
# 데이터셋 클래스 정의
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x): 
        toks = self.tokenizer(x, truncation=True, padding='max_length', max_length=256, return_tensors="pt")['input_ids']
        print(f"Tokenized: {toks.shape} - {toks}")
        return toks.squeeze(0)


    def decodes(self, x):
        return self.tokenizer.decode(x, skip_special_tokens=True)

In [28]:
class TransformersTokenizer(Transform):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encodes(self, x): 
        # Tokenizing input text with padding and truncation
        encodings = self.tokenizer(x, truncation=True, padding='max_length', max_length=256, return_tensors="pt")
        
        # Extracting input_ids and attention_mask
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        
        print(f"Tokenized: {input_ids.shape} - {input_ids}")
        return input_ids.squeeze(0), attention_mask.squeeze(0)

    def decodes(self, x):
        return self.tokenizer.decode(x, skip_special_tokens=True)


In [29]:
## 용량? 이슈로 잠시 조정..
data = data[:int(len(data)*0.3)]

In [30]:
# 텍스트 데이터를 리스트 형태로 변환
train = data['input_text'][:int(len(data)*0.9)].tolist()
test = data['input_text'][int(len(data)*0.9):].tolist()

splits = [[0],[1]]

# DataLoader 초기화
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 4, 128
dls = tls.dataloaders(bs=batch, seq_len=seq_len)

Tokenized: torch.Size([100, 256]) - tensor([[ 9130,   739,   466,  ..., 10912,   387, 12187],
        [ 9085,   739,   466,  ...,  7198,   389,  9524],
        [ 9085,   739,   466,  ...,  8137, 17828,  9016],
        ...,
        [ 9130,   739,   466,  ...,  9194,   777,  7237],
        [ 9085,   739,   466,  ..., 16897, 40054, 19037],
        [ 9085,   739,   466,  ...,  9322, 37970,  9124]])
Tokenized: torch.Size([100, 256]) - tensor([[ 9130,   739,   466,  ..., 10912,   387, 12187],
        [ 9085,   739,   466,  ...,  7198,   389,  9524],
        [ 9085,   739,   466,  ...,  8137, 17828,  9016],
        ...,
        [ 9130,   739,   466,  ...,  9194,   777,  7237],
        [ 9085,   739,   466,  ..., 16897, 40054, 19037],
        [ 9085,   739,   466,  ...,  9322, 37970,  9124]])
Tokenized: torch.Size([12, 256]) - tensor([[ 9085,   739,   466,  ..., 21154, 12187,  9194],
        [ 9085,   739,   466,  ..., 10510,  7470,  7235],
        [ 9085,   739,   466,  ...,  8269, 40823,   7

In [58]:
from fastai.text.all import Learner, CrossEntropyLossFlat, Perplexity, Callback

# DataLoader와 모델 확인 후, Learner 초기화
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=Perplexity())

# 모델 fine-tune
learn.fine_tune(1)

epoch,train_loss,valid_loss,perplexity,time


: 

- test

In [56]:
# 감정1과 감정2를 숫자로 변환
# LabelEncoder 초기화
le = LabelEncoder()
le.fit(emotions)

lyrics_final['감정1_encoded'] = le.transform(lyrics_final['top_sentiment'])
lyrics_final['감정2_encoded'] = le.transform(lyrics_final['second_sentiment'])


In [39]:
lyrics_final['감정1_encoded'][3]

np.int64(0)

In [57]:
## 숫자 고르기
i = 3

#final = lyrics_final[3]

lyrics = " ".join(lyrics_final["translated_lyrics"][i])

# 'translated_lyrics' 컬럼의 각 리스트 요소를 하나의 문자열로 합침
prompt = lyrics_final['감정1_encoded'][i].astype(str) + " | " + lyrics_final['감정2_encoded'][i].astype(str) + " | " + lyrics
prompt

'0 | 3 | 내 가슴속에 사는 사람 내가 그토록 아끼는 사람  너무 소중해 마음껏 안아보지도 못했던  누구에게나 흔한 행복 한 번도 준적이 없어서  맘 놓고 웃어본 적도 없는 그댈 사랑합니다  내가 기쁠 때나 슬플 때나 함께 울고 웃어주던  그댈 위해 내가 할 수 있는건 뭐든 해주고 싶어  안녕 내 사랑 그대여 이젠 내가 지켜줄게요  못난 날 믿고 참고 기다려줘서 고마워요  안녕 내 사랑 그대여 영원토록 사랑할게요  다시 태어나서 사랑한대도 그대이고 싶어요  어두운 밤길을 잃어도 서로 등불이 되어주고  비바람 몰아쳐도 지금 잡은 두 손 놓지 말아요  내가 힘들 때나 아플 때나 내 곁에 있어준 그대  미안하단 말로 고맙단 말을 대신 하던 나였죠  안녕 내 사랑 그대여 이젠 내가 지켜줄게요  못난 날 믿고 참고 기다려줘서 고마워요  안녕 내 사랑 그대여 영원토록 사랑할게요  다시 태어나서 사랑한대도 그대이고 싶어요  넘어지고 몇 번을 다시 넘어진다 해도  그대만 있다면 다시 일어날 수 있는데  < 간주    안녕 내 사랑 그대여 이젠 내가 지켜줄게요  못난 날 믿고 참고 기다려줘서 고마워요  안녕 내 사랑 그대여 이렇게 그댈 부를 때면  너무 행복해서 눈물이 나죠 사랑해요  < 끝입니다. 합창단'

In [ ]:
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=256,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,       
                           use_cache=True
                          ) 
tokenizer.decode(preds[0].cpu().numpy())